# oneM2M - Basic Resources and Interactions
This notebook shows the basic interactions with a CSE using REST calls. Examples include:

- Create an &lt;AE> resource
- Create a &lt;Container> resource
- Create one or more &lt;ContentInstance> resources
- Retrieve the latest &lt;ContentInstance> resource
- Update the &lt;Container> resource
- Retrieve the &lt;Container> resource

## Intitialization
The section does import necessary modules and configurations.  
**Please execute it only once when the notebook (re)started.**

In [1]:
%run init.py

<span style='color:green'>**Configuration Ready**</span>

## Create an &lt;AE> Resource
This example registers a new &lt;AE> resource in the CSE. &lt;AE>'s represent applications or business services in a oneM2M system.

Creating this (or other) resource is done using a POST request together with a couple of parameters and mandatory attributes:

### Parameters

### Resource Attributes

# - **api** : Application Identifier. An Application Identifier uniquely identifies an M2M Application in a given context.
#- **rr** : Request Reachability. This attribute indicates whether a resource can receive requests.
#- **srv** : Supported Release Versions. An array that specifies the supported oneM2M specification releases.

Note, that the request target is the &lt;CSEBase> resource. All create requests target a parent resource.

In [2]:
CREATE (                                      # CREATE request
    # Target on the CSE
    target = cseRN, 
    
    # Request Parameters
    parameters = {
        From                    : 'C',        # Request's originator
        RequestIdentifier       : '123',      # Request identifier,
        ReleaseVersionIndicator : '3',        # Request's release version indicator
        ResourceType            : '2'         # The request creates an <AE> resource
    },
    
    # Request Content
    content = {   
        "m2m:ae": {
            "rn": "Notebook-AE",              # Name of the resource instance
            "api": "NTestAE",                 # Non-registered App-ID
            "rr": True,                       # Request reachability
            "srv": [ "3" ]                    # Supported Release Versions
        }
    }
)

---
### HTTP Request

**http://localhost:8080/cse-in**


| HTTP Header | oneM2M Parameter | Value |
|:---|:---|:---|
| [X-M2M-Origin](#_blank "Originator - The request's 'From' parameter") | From | C |
| [X-M2M-RI](#_blank "Request Identifier - Used to uniquely identify a request") | Request Identifier | 123 |
| [X-M2M-RVI](#_blank "Release Version Indicator - The requester indicates the release version of the oneM2M specification") | Release Version Indicator | 3 |
| [Accept](#_blank "The Originator may use the Accept header to indicate which media types are acceptable for the response") |  | application/json |
| [Content-Type](#_blank "Any HTTP request or response with content shall include the Content-type header set to one of 'application/xml', 'application/json', or the oneM2M defined media types") |  | application/json;ty=2 |



#### Request Content


<span style='font-family: monospace;'>{</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:ae](#_blank "ApplicationEntity
An entity in the application layer that implements an M2M application service logic")": {</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[rn](#_blank "resource name
jhjhjk dksjekdjf kdsj dskjfdksjf dksjfkds jlksdjkds lkdjfkdsjf kkdjkds eoiui kdjkdsjkdjf sjahdsjhdsj ksjdkdsj jdksf...")": "Notebook-AE",</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[api](#_blank "App-ID
An Application Identifier uniquely identifies an M2M Application in a given context. It starts with R (registered) or N (non-registered)")": "NTestAE",</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[rr](#_blank "request reachability
This attribute indicates whether a resource can receive requests")": true,</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[srv](#_blank "supported release versions
An array that specifies the supported oneM2M specification releases")": [</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"3"</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;]</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;}</span>  
<span style='font-family: monospace;'>}</span>  


...

<span style='color:red'>409 (CONFLICT)</span>


| HTTP Header | oneM2M Parameter | Value |
|:---|:---|:---|
| [X-M2M-RSC](#_blank "Result Status Code - The request's extended result status code") | Result Status Code | [4105](#_blank "Conflict") |
| [X-M2M-RI](#_blank "Request Identifier - Used to uniquely identify a request") | Request Identifier | 123 |
| [X-M2M-RVI](#_blank "Release Version Indicator - The requester indicates the release version of the oneM2M specification") | Release Version Indicator | 3 |
| [Content-Type](#_blank "Any HTTP request or response with content shall include the Content-type header set to one of 'application/xml', 'application/json', or the oneM2M defined media types") |  | application/json |
| [Date](#_blank "The response's date in UTC/GMT") |  | Mon, 22 Feb 2021 20:41:35 GMT |



**Result Content**


<span style='font-family: monospace;'>{</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>"[m2m:dbg](#_blank "debug information and error messages")": "Resource already exists"</span></span>  
<span style='font-family: monospace;'>}</span>  


---

The response introduces some new attributes:

- **pi** : This is the identifier of the parent resource.
- **et** : Expiration time/date after which the CSE will delete the resource.
- **aei** : An Application Entity Identifier uniquely identifies an AE globally.

**Note**: If you see an error "409" or "Name already present" then don't worry. It just means that an &lt;AE> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.

## Create a &lt;Container> Resource
In this section we add a &lt;Container> resource to the &lt;AE> resource. A &lt;Container> represents a data point that can hold a configurable number of dsta instances. A &lt;Container> may also hold a sub-containers.

If created with no further attributes, the CSE will assign defaults to the &lt;Container> resource.

In [3]:
CREATE (                                           # CREATE request
    f'{cseRN}/Notebook-AE',
    
    # Request Parameters
    parameters = {
        'X-M2M-Origin' : originator,               # Request's originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON serialized
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, request creates a <container> resource
    },
    
    # Request Content
    content = {
        "m2m:cnt": {
            "rn":"Container"
        }
    }
)

---
### HTTP Request

**http://localhost:8080/cse-in/Notebook-AE**


| HTTP Header | oneM2M Primite Parameter | Value |
|:---|:---|:---|
| [X-M2M-Origin](#_blank "Originator - The request's 'From' parameter") | tFrom | CAdmin |
| [X-M2M-RI](#_blank "Request Identifier - Used to uniquely identify a request") | tRequest Identifier | 0 |
| [Accept](#_blank "The Originator may use the Accept header to indicate which media types are acceptable for the response") | t | application/json |
| [Content-Type](#_blank "Any HTTP request or response with content shall include the Content-type header set to one of 'application/xml', 'application/json', or the oneM2M defined media types") | t | application/json |



#### Request Content


<span style='font-family: monospace;'>{</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:cnt](#_blank "container")": {</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[rn](#_blank "resource name
jhjhjk dksjekdjf kdsj dskjfdksjf dksjfkds jlksdjkds lkdjfkdsjf kkdjkds eoiui kdjkdsjkdjf sjahdsjhdsj ksjdkdsj jdksf...")": "Container"</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;}</span>  
<span style='font-family: monospace;'>}</span>  


...

<span style='color:red'>400 (BAD REQUEST)</span>


| HTTP Header | oneM2M Primite Parameter | Value |
|:---|:---|:---|
| [X-M2M-RSC](#_blank "Result Status Code - The request's extended result status code") | tResult Status Code | [4000](#_blank "Bad Request") |
| [X-M2M-RI](#_blank "Request Identifier - Used to uniquely identify a request") | tRequest Identifier | 0 |
| [X-M2M-RVI](#_blank "Release Version Indicator - The requester indicates the release version of the oneM2M specification") | tRelease Version Indicator | None |
| [Content-Type](#_blank "Any HTTP request or response with content shall include the Content-type header set to one of 'application/xml', 'application/json', or the oneM2M defined media types") | t | application/json |
| [Date](#_blank "The response's date in UTC/GMT") | t | Mon, 22 Feb 2021 19:17:47 GMT |



**Result Content**


<span style='font-family: monospace;'>{</span>  
<span style='font-family: monospace;'>&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>"[m2m:dbg](#_blank "debug information and error messages")": "Mandatory Release Version Indicator is missing in request"</span></span>  
<span style='font-family: monospace;'>}</span>  


---

The following attributes are used with the &lt;Container> resource:

- **st** : The State Tag is incremented every time the resource is modified.
- **mni** : Maximum number of direct data instances in the &lt;Container> resource.
- **mbs** : Maximum size in bytes of data.
- **mia** : Maximum age of a direct data instances in the &lt;Container> resource.
- **cni** : Current number of direct data instances in the &lt;Container> resource.
- **cbs** : Current size in bytes of data.
- **ol** : Resource identifier of a virtual resource that points to the oldest data instance of the &lt;Container> resource.
- **la** : Resource identifier of a virtual resource that points to the latest data instance of the &lt;Container> resource.

**Note**: If you see an error "409" or "Name already present" then don't worry. It just means that an &lt;Container> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.

## Add a &lt;ContentInstance> to the &lt;Container>

Now, we add an actual value to the *myContainer* &lt;Container>. 

These attributes are part of the request:

- **cnf** : This specifies the content format. It specifies the media type as well as an encoding type.
- **con** : This is the actual content (ie. the value) that will be stored in the &lt;Container resource. It must contain media information and may optionally specify an optional content encoding (here 0 means "plain, no transfer encoding"), and content security.

&lt;ContentInstance>'s can only be added and read, but not updated or deleted.

**Note**: You can execute the following code as often as you like in order to create more &lt;ContentInstance> resources.

In [3]:
CREATE (                                           # CREATE request
    url + '/Notebook-AE/Container',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
    },
    
    # Request Body
    {
        "m2m:cin": {
            "cnf": "text/plain:0",
            "con": "Hello, World!"
        }
    }
)

---
### Request to "http://localhost:8080/cse-in/Notebook-AE/Container"


| Parameter | Value |
|:---|:---|
| X-M2M-Origin | CAdmin |
| X-M2M-RI | 0 |
| Accept | application/json |
| Content-Type | application/json;ty=4 |



#### Request Content


{  
&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:cin](#_blank "content instance")": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cnf](#_blank "content info")": "text/plain:0",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[con](#_blank "content")": "Hello, World!"  
&nbsp;&nbsp;&nbsp;&nbsp;}  
}  


---
### Response

<span style='color:red'>404 (NOT FOUND)</span>


| Parameter | Value |
|:---|:---|
| Server | ACME 0.7.0-dev |
| [X-M2M-RSC](#_blank "The request's Result Status Code : It provides a status code about the operation") | [4004](#_blank "Not Found") |
| X-M2M-RI | 0 |
| X-M2M-RVI | None |
| Content-Type | application/json |
| Content-Length | 40 |
| Date | Sun, 21 Feb 2021 23:27:54 GMT |



**Content**


{  
&nbsp;&nbsp;&nbsp;&nbsp;<span style='color:red'>"[m2m:dbg](#_blank "debug information")": "parent resource not found"</span>  
}  


---

A new attribute:

- **cs** : This attribute contains the size of the content of the **con** attribute.

## Retrieve the latest &lt;ContentInstance> resource
This request will retrieve the latest data instance from the &lt;Container>.

In [11]:
RETRIEVE (                                          # RETRIEVE request
    url + '/Notebook-AE/Container/la',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,                # Set the originator
        'X-M2M-RI'     : '0',                       # Unique request identifier
        'Accept'       : 'application/json'         # Response shall be JSON    
    }
)

Sending request to **http://localhost:8080/cse-in/Notebook-AE/Container/la**


**Headers**



| Header Field | Value |
|:---|:---|
| X-M2M-Origin | CAdmin |
| X-M2M-RI | 0 |
| Accept | application/json |


---
### Response

<span style='color:green'>200 (OK)</span>


**Headers**



| Header Field | Value |
|:---|:---|
| Server | ACME 0.7.0-dev |
| [X-M2M-RSC](#_blank "The request's Result Status Code : It provides a status code about the operation") | [2000](#_blank "OK - The request succeeded") |
| X-M2M-RI | 0 |
| X-M2M-RVI | None |
| Content-Type | application/json |
| Content-Length | 272 |
| Date | Sun, 21 Feb 2021 21:48:13 GMT |



**Body**


{  
&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:cin](#_blank "content instance")": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cnf](#_blank "content info")": "text/plain:0",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[con](#_blank "content")": "Hello, World!",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ri](#_blank "resource identifier")": "cin5052874453861657402",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[rn](#_blank "resource name  
jhjhjk dksjekdjf kdsj dskjfdksjf dksjfkds jlksdjkds lkdjfkdsjf kkdjkds eoiui kdjkdsjkdjf sjahdsjhdsj ksjdkdsj jdksf...")": "cin_W1l6npXJJy",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ct](#_blank "creation time")": "20210221T183218,425817",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[lt](#_blank "last modified time")": "20210221T183218,425817",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[et](#_blank "expiration time")": "20220221T183218,425853",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[pi](#_blank "parent identifier")": "cnt3972778165262898416",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[st](#_blank "state tag")": 1,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ty](#_blank "resource type")": 4,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cs](#_blank "content size")": 13  
&nbsp;&nbsp;&nbsp;&nbsp;}  
}  


---

## Update the &lt;Container> Resource

With this request we will set the *MinimumNumberOfInstances* (**mni**) attribute to a new value.

In [6]:
UPDATE (                                           # UPDATE request
    url + '/Notebook-AE/Container',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
    },
    
    # Request Body
    {
        "m2m:cnt": {
            "mni": 10001
        }
    }
)

Sending request to **http://localhost:8080/cse-in/Notebook-AE/Container**


**Headers**



| Header Field | Value |
|:---|:---|
| X-M2M-Origin | CAdmin |
| X-M2M-RI | 0 |
| Accept | application/json |
| Content-Type | application/json;ty=3 |



**Body**


{  
&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:cnt](#_blank "container")": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[mni](#_blank "max number of instances")": 10001  
&nbsp;&nbsp;&nbsp;&nbsp;}  
}  


...


**Headers**



| Header Field | Value |
|:---|:---|
| Server | ACME 0.7.0-dev |
| [X-M2M-RSC](#_blank "The request's Result Status Code : It provides a status code about the operation") | [2004](#_blank "Updated") |
| X-M2M-RI | 0 |
| X-M2M-RVI | None |
| Content-Type | application/json |
| Content-Length | 248 |
| Date | Sun, 21 Feb 2021 18:32:18 GMT |



**Body**


{  
&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:cnt](#_blank "container")": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[rn](#_blank "resource name  
jhjhjk dksjekdjf kdsj dskjfdksjf dksjfkds jlksdjkds lkdjfkdsjf kkdjkds eoiui kdjkdsjkdjf sjahdsjhdsj ksjdkdsj jdksf...")": "Container",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ri](#_blank "resource identifier")": "cnt3972778165262898416",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ct](#_blank "creation time")": "20210221T183218,318605",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[lt](#_blank "last modified time")": "20210221T183218,588477",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[et](#_blank "expiration time")": "20220221T183218,318643",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[pi](#_blank "parent identifier")": "CePke89JxyK",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[st](#_blank "state tag")": 2,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ty](#_blank "resource type")": 3,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[mni](#_blank "max number of instances")": 10001,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[mbs](#_blank "max byte size")": 10000,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cni](#_blank "current number of instances")": 1,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cbs](#_blank "current byte size")": 13  
&nbsp;&nbsp;&nbsp;&nbsp;}  
}  


---

The CSE returns the resource. Also note the change of the *lastModificationTime* (lt) and *status* (st) attributes.

## Check the &lt;Container> resource
Retrieve the &lt;Container> resource to see all the changes and its current state.

In [7]:
RETRIEVE (                                          # RETRIEVE request
    url + '/Notebook-AE/Container',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,                # Set the originator
        'X-M2M-RI'     : '0',                       # Unique request identifier
        'Accept'       : 'application/json'         # Response shall be JSON    
    }
)

Sending request to **http://localhost:8080/cse-in/Notebook-AE/Container**


**Headers**



| Header Field | Value |
|:---|:---|
| X-M2M-Origin | CAdmin |
| X-M2M-RI | 0 |
| Accept | application/json |


---
### Response

<span style='color:green'>200 (OK)</span>


**Headers**



| Header Field | Value |
|:---|:---|
| Server | ACME 0.7.0-dev |
| [X-M2M-RSC](#_blank "The request's Result Status Code : It provides a status code about the operation") | [2000](#_blank "OK - The request succeeded") |
| X-M2M-RI | 0 |
| X-M2M-RVI | None |
| Content-Type | application/json |
| Content-Length | 248 |
| Date | Sun, 21 Feb 2021 18:32:18 GMT |



**Body**


{  
&nbsp;&nbsp;&nbsp;&nbsp;"[m2m:cnt](#_blank "container")": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[rn](#_blank "resource name  
jhjhjk dksjekdjf kdsj dskjfdksjf dksjfkds jlksdjkds lkdjfkdsjf kkdjkds eoiui kdjkdsjkdjf sjahdsjhdsj ksjdkdsj jdksf...")": "Container",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ri](#_blank "resource identifier")": "cnt3972778165262898416",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ct](#_blank "creation time")": "20210221T183218,318605",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[lt](#_blank "last modified time")": "20210221T183218,588477",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[et](#_blank "expiration time")": "20220221T183218,318643",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[pi](#_blank "parent identifier")": "CePke89JxyK",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[st](#_blank "state tag")": 2,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[ty](#_blank "resource type")": 3,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[mni](#_blank "max number of instances")": 10001,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[mbs](#_blank "max byte size")": 10000,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cni](#_blank "current number of instances")": 1,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"[cbs](#_blank "current byte size")": 13  
&nbsp;&nbsp;&nbsp;&nbsp;}  
}  


---

The CSE returns the resource.